# Setup

## Ecoscope

In [ ]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import sys

## Google EarthEngine

Change it to your own credentials (project name)

In [ ]:
import ee
import os

ee.Authenticate()  # will prompt for the Cloud project
ee.Initialize(project="PROJECT_NAME") # Add your project name

## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
WORKING_DIRECTORY = '/content/drive/My Drive/TIF_TEST' # Change for custom folder path to save rasters from EarthEngine

os.chdir(WORKING_DIRECTORY)

### Extracting TIFs

In [ ]:
df_metadata = pd.read_csv("metadata/hotspots_za_counts_merged_1km_trimmed.csv") # Change for custom filepath with hotspots coords and polygon

In [ ]:
len(df_metadata)

In [ ]:
tif_files = [os.path.splitext(f)[0] for f in os.listdir(".") if f.lower().endswith(".tif")]
len(tif_files)

In [ ]:
import ee
import geopandas as gpd
from shapely import wkt

# 1. Load your polygon (WKT or shapefile)
# Option 1: from WKT

for i in range(len(df_metadata)):
  row = df_metadata.iloc[i]
  hotspot_name = row['hotspot_id']
  if hotspot_name not in tif_files:
    print(hotspot_name)

In [ ]:
import ee
import geopandas as gpd
from shapely import wkt

for i in range(len(df_metadata)):
  row = df_metadata.iloc[i]
  hotspot_name = row['hotspot_id']
  if hotspot_name not in tif_files:
    polygon_string = row['geometry']

    wkt_string = wkt.loads(polygon_string).exterior.coords
    print(list(wkt_string))
    geometry = ee.Geometry.Polygon(list(wkt_string))

    # 2. Get Sentinel-2 image collection and filter
    s2 = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(geometry)
        .filterDate("2024-05-01", "2024-09-01")
        .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 10))
    )


    # 4. Mosaic and select RGB + NIR bands
    mosaic = s2.mosaic().select(["B2", "B3", "B4", "B8"])

    # 5. Export to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=mosaic,
        description="Sentinel2_RGBNIR_Mosaic",
        folder="TIF_TEST", # Change for custom folder name
        fileNamePrefix=hotspot_name,
        region=geometry,
        scale=10,
        maxPixels=1e13
    )

    task.start()
    print("Export started. Check your Google Drive.")
    break

In [ ]:
task.status()
